<a href="https://www.kaggle.com/code/peacehegemony/deutsche-nationalbibliografie?scriptVersionId=118520622" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Start by installing any necessary libraries. For example, if you want to work with RDF data, you may want to install the rdflib library.

In [1]:
%%capture
!pip install rdflib

Next, download the data dump files from the URLs listed in the "resources" section of the JSON file. For example, if the first resource listed is an RDF/XML file, you could download it using the wget command:

In [2]:
%%capture
!wget https://data.dnb.de/opendata/authorities-gnd-sachbegriff_lds.ttl.gz
!gzip -d -f authorities-gnd-sachbegriff_lds.ttl.gz
# !echo y | gzip -d DNBTitel.ttl.gzauthorities-gnd-sachbegriff_lds.ttl.gz
# !yes n | gzip -d authorities-gnd-sachbegriff_lds.ttl.gz

Once you have downloaded the data dump files, you can load them into your notebook using the appropriate libraries. For example, if you downloaded an RDF/XML file, you could use the rdflib library to parse the file and create a graph object:

In [3]:
from rdflib import Graph



g = Graph()
g.parse('/kaggle/working/authorities-gnd-sachbegriff_lds.ttl', format='turtle')

<Graph identifier=N1ff809e2a5e6429e9b94d3ae7d27cbf9 (<class 'rdflib.graph.Graph'>)>

After loading the data, you can start exploring it by counting the number of triples, for example:

In [4]:
print(len(g))

2696851


Extracting specific information: Using SPARQL queries, you can extract specific information from the data. For example, you can find the number of resources of a specific type.

In [5]:
from rdflib.plugins.sparql import prepareQuery



q = prepareQuery('''
    SELECT (COUNT(?s) AS ?count)
    WHERE {
        ?s a <http://example.com/vocab/ResourceType> .
    }
''', initNs={'ex': 'http://example.com/vocab/'})


res = g.query(q)

for row in res:
    print(row.count)

<built-in method count of ResultRow object at 0x7f2937f2a460>


In [6]:
from rdflib import Namespace, RDF
# from rdflib.plugins.sparql import prepareQuery



# g = Graph().parse('/kaggle/working/authorities-gnd-sachbegriff_lds.ttl', format='turtle')



# Define the namespaces used in the file
gndo = Namespace("https://d-nb.info/standards/elementset/gnd#")
dcterms = Namespace("http://purl.org/dc/terms/")


# Find the description level of the resource with identifier "4000002-3"
q = prepareQuery('''
    SELECT ?level 
    WHERE { 
        <https://d-nb.info/gnd/4000002-3> dcterms:describedby ?desc .
        ?desc gndo:descriptionLevel ?level
    }
''', initNs={'gndo':gndo, 'dcterms':dcterms})

results = g.query(q)
for r in results:
    print(r.level)

    
# Find all resources with a certain description level
q = prepareQuery('''
    SELECT ?res
    WHERE {
        ?res dcterms:describedby ?desc .
        ?desc gndo:descriptionLevel <https://d-nb.info/standards/vocab/gnd/description-level#1> .
    }
''', initNs={'gndo':gndo, 'dcterms':dcterms})

results = g.query(q)

Generating a network of relationships: You can also use the graph to generate a network of relationships between the resources. For example, you can use the NetworkX library to create a graph object and then use the draw_networkx function to visualize the graph.

In [7]:
import networkx as nx
import matplotlib.pyplot as plt



# plt.rcParams['figure.figsize'] = [256, 256]
# plt.rcParams['figure.dpi'] = 300


# G = nx.DiGraph()
# for s, p, o in g:
#     G.add_edge(s, o)
# nx.draw(G, with_labels=True)
# plt.show()

Analyze the distribution of predicates: You can also use the collections library to count the number of occurrences of each predicate in the graph and visualize the results using a bar chart.

In [8]:
from collections import Counter



plt.figure(figsize = (256, 18), dpi = 150)


predicates = [p for s, p, o in g]
predicate_counts = Counter(predicates)


plt.bar(predicate_counts.keys(), predicate_counts.values())
plt.xlabel('Predicate')
plt.ylabel('Count')

plt.show()